# Import

In [199]:
import xml.etree.ElementTree as ET
import pandas as pd
import matplotlib.pyplot as plt

# Transformation

## Use `xml.etree.ElementTree` module parse data

In [200]:
def convert_xml_to_dataframe(file_path):
    '''
    convert xml file into a dataframe format
    '''
    tree = ET.parse(file_path)
    root = tree.getroot()
    # parse scenatio dates
    date_list = []
    for date in root.iter('scenarioDates'):
        date_list.append(date.text.split())
    # dimensionality
    date_list = date_list[0]
    # parse desk names
    desk_list = []
    for desk in root.iter('value'):
        if 'DESK' in desk.text:
            desk_list.append(desk.text.split())
    # dimensionality
    desk_list = [desk_list[i][0] for i in range(len(desk_list))]
    # parst risk class names
    risk_class_list = []
    for risk_class in root.iter('pvRiskClassList'):
        risk_class_list.append(risk_class.text.split())
    # dimensionality
    risk_class_list = risk_class_list[0]
    # combine desk names with risk class names
    desk_class_list = []
    for desk in desk_list:
        for risk_class in risk_class_list:
            desk_class_list.append(str(desk) + '_' + str(risk_class))
    # parst pv value
    pv_list = []
    for pv in root.iter('pvList'):
        pv_list.append(pv.text.split())
    # create a dataframe and change data type to float
    PV = pd.DataFrame(pv_list, dtype=float).T
    PV.columns = desk_class_list
    PV = PV.sort_index(axis=1)
    PV.insert(0, 'DATE', date_list)
    return PV

# Read data

In [201]:
sp_pv = convert_xml_to_dataframe('./Desktop/0905_sp.xml')

In [202]:
sp_pv.head()

,DATE,DESK3_A_TOTAL,DESK3_B_TOTAL,DESK3_C_TOTAL,DESK3_D_TOTAL,DESK3_E_TOTAL,DESK3_F_TOTAL,DESK3_G_TOTAL,DESK3_G_TOTAL,DESK3_H_TOTAL,DESK3_I_TOTAL,DESK3_I_TOTAL,DESK3_J_TOTAL,DESK3_J_TOTAL,DESK3_K_TOTAL,DESK3_K_TOTAL,DESK3_L_TOTAL,DESK3_M_TOTAL,DESK3_N_TOTAL
0,2006-09-01,-112087.430020,39992.748489,-29403.202819,-488603.263931,-420233.666421,159978.873468,78060.640130,-80238.715858,-8010.202600,-201749.761901,204717.517291,466239.756682,-473452.656340,2.470208e+06,-2.507963e+06,69532.114909,-9692.277600,9692.277600
1,2006-09-04,-77098.452860,5029.497488,5683.991324,-491266.561093,-421789.621048,153103.169176,73672.664706,-75970.966053,-8075.844326,-197504.788279,200303.623304,470609.939101,-477727.772264,2.474954e+06,-2.511822e+06,30133.181717,-9757.898901,9757.898901
2,2006-09-05,-56047.687106,-16834.440500,27293.176441,-469116.626155,-417671.663216,197966.269708,115931.979133,-117508.673575,-7946.013583,-238818.608379,242343.489429,428065.330039,-434512.894633,2.428717e+06,-2.465082e+06,6059.221657,-9628.051141,9628.051141
3,2006-09-06,-55147.727131,-16780.072178,27254.685747,-490186.348545,-417044.924967,159317.478409,84694.500835,-86865.284038,-8014.780511,-208335.356201,211226.947883,459428.914399,-466276.978556,2.462722e+06,-2.498789e+06,6025.633860,-9696.830808,9696.830808
4,2006-09-07,-47136.796232,-24503.419640,34669.235295,-490805.870357,-426383.027586,150481.569742,73004.461961,-75347.175327,-8011.241820,-196850.165257,199567.979523,471095.065988,-478102.310521,2.474931e+06,-2.511113e+06,-2280.266697,-9693.298276,9693.298276


# Get Stress Period

In [209]:
# get profit and lost by subtracing the next day's present value
sp_pl = sp_pv.iloc[:, 1:].diff(periods=-1)
sp_pl.insert(0, "DATE", sp_pv.DATE)
sp_pl = sp_pl[:-1]

In [210]:
sp_pl.head()

,DATE,DESK3_A_TOTAL,DESK3_B_TOTAL,DESK3_C_TOTAL,DESK3_D_TOTAL,DESK3_E_TOTAL,DESK3_F_TOTAL,DESK3_G_TOTAL,DESK3_G_TOTAL,DESK3_H_TOTAL,DESK3_I_TOTAL,DESK3_I_TOTAL,DESK3_J_TOTAL,DESK3_J_TOTAL,DESK3_K_TOTAL,DESK3_K_TOTAL,DESK3_L_TOTAL,DESK3_M_TOTAL,DESK3_N_TOTAL
0,2006-09-01,-34988.977161,34963.251001,-35087.194143,2663.297162,1555.954627,6875.704292,4387.975424,-4267.749805,65.641727,-4244.973622,4413.893988,-4370.182419,4275.115925,-4745.808809,3858.781738,39398.933192,65.621301,-65.621301
1,2006-09-04,-21050.765753,21863.937989,-21609.185117,-22149.934938,-4117.957833,-44863.100531,-42259.314426,41537.707522,-129.830743,41313.820100,-42039.866126,42544.609062,-43214.877632,46236.976004,-46739.912169,24073.960060,-129.847760,129.847760
2,2006-09-05,-899.959975,-54.368322,38.490694,21069.722390,-626.738248,38648.791298,31237.478298,-30643.389537,68.766928,-30483.252177,31116.541546,-31363.584361,31764.083923,-34004.580969,33706.711062,33.587797,68.779667,-68.779667
3,2006-09-06,-8010.930899,7723.347462,-7414.549549,619.521812,9338.102619,8835.908667,11690.038874,-11518.108711,-3.538691,-11485.190945,11658.968360,-11666.151589,11825.331965,-12209.033523,12324.422776,8305.900558,-3.532533,3.532533
4,2006-09-07,-47041.517423,47153.818046,-47166.459769,2098.994726,-946.907929,-13616.693197,-9489.929933,9193.939953,-37.912011,9244.043806,-9541.520301,8968.605028,-9265.285411,8142.288709,-8436.580728,52820.688631,-37.921006,37.921006


In [256]:
# sum each 250 days period profit and lost
sp_pl_sum = sp_pl.iloc[:, 1:].rolling(250).sum()
sp_pl_sum.insert(0, "DATE", sp_pl.DATE)
sp_pl_sum = sp_pl_sum[250:]
sp_pl_sum.head()

,DATE,DESK3_A_TOTAL,DESK3_B_TOTAL,DESK3_C_TOTAL,DESK3_D_TOTAL,DESK3_E_TOTAL,DESK3_F_TOTAL,DESK3_G_TOTAL,DESK3_G_TOTAL,DESK3_H_TOTAL,DESK3_I_TOTAL,DESK3_I_TOTAL,DESK3_J_TOTAL,DESK3_J_TOTAL,DESK3_K_TOTAL,DESK3_K_TOTAL,DESK3_L_TOTAL,DESK3_M_TOTAL,DESK3_N_TOTAL
250,2007-08-21,45084.848941,-42861.806882,38571.770710,-34605.259438,-34804.889346,-12164.280396,-34682.184039,34481.822515,-2747.418379,34126.265428,-34286.050330,36080.889425,-36407.104206,41944.761425,-42770.265862,-46296.942952,-2747.880835,2747.880835
251,2007-08-22,41311.225286,-41028.980575,44384.897617,8080.062241,-4361.911185,49272.354739,34510.429260,-33761.873964,-174.253150,-33635.241021,34482.498250,-34331.387461,34632.051720,-36419.826831,35080.712131,-49939.689262,-174.291651,174.291651
252,2007-08-23,97497.132251,-98187.346907,97204.123765,19592.169351,29303.713699,28213.051472,35512.584561,-34942.215146,-131.547965,-34797.854830,35334.342107,-35591.454928,36288.721481,-37972.255270,39151.859623,-109090.959249,-131.559727,131.559727
253,2007-08-24,28926.062896,-27895.002515,28549.614067,23170.620049,23377.486370,-2182.128408,15855.727014,-15903.067456,219.814907,-15645.940085,15574.534253,-17059.460905,17080.149614,-21300.023371,21596.995723,-31750.632420,219.790305,-219.790305
254,2007-08-27,59453.012586,-59725.034983,60448.318757,27415.596424,30233.638141,17765.385989,32956.937683,-32638.598392,858.134833,-32388.895769,32626.691068,-33761.600045,34394.959823,-37879.712904,39699.766107,-66790.833370,858.271899,-858.271899


In [257]:
# combine duplicated columns
sp_pl_sum = sp_pl_sum.groupby(sp_pl_sum.columns, axis=1).sum()
# get the minimum value list
sp_pl_sum_min = sp_pl_sum.min()[1:]
sp_pl_sum_min

DESK3_A_TOTAL   -3.15923e+06
DESK3_B_TOTAL   -3.13586e+06
DESK3_C_TOTAL   -3.12931e+06
DESK3_D_TOTAL        -240682
DESK3_E_TOTAL        -227322
DESK3_F_TOTAL        -263693
DESK3_G_TOTAL       -6159.57
DESK3_H_TOTAL       -10974.8
DESK3_I_TOTAL       -7082.66
DESK3_J_TOTAL       -7290.27
DESK3_K_TOTAL       -33055.5
DESK3_L_TOTAL   -3.50771e+06
DESK3_M_TOTAL       -10975.6
DESK3_N_TOTAL       -12831.1
dtype: object

In [269]:
# find stress period end date
count_desk = 0
end_date_index = []
sp_end_date = []
for count_desk in range(sp_pl_sum_min.size):
    index = sp_pl_sum[sp_pl_sum[sp_pl_sum_min.index[count_desk]]== sp_pl_sum_min.values[count_desk]].index[0]
    end_date_index.append(index)
    sp_end_date.append(sp_pl_sum.iloc[index].DATE)
sp_end_date

['2009-04-14',
 '2009-04-13',
 '2009-04-14',
 '2009-01-30',
 '2009-01-30',
 '2010-01-06',
 '2010-01-06',
 '2010-09-13',
 '2010-01-06',
 '2009-07-22',
 '2009-07-22',
 '2009-04-13',
 '2010-09-13',
 '2009-09-24']

In [271]:
# find stress period start date
sp_start_date = []
for i in range(len(end_date_index)):
    sp_start_date.append(sp_pl_sum.iloc[end_date_index[i]-250].DATE)
sp_start_date

['2008-04-25',
 '2008-04-24',
 '2008-04-25',
 '2008-02-13',
 '2008-02-13',
 '2009-01-19',
 '2009-01-19',
 '2009-09-24',
 '2009-01-19',
 '2008-08-04',
 '2008-08-04',
 '2008-04-24',
 '2009-09-24',
 '2008-10-07']

In [272]:
# store into dataframe
stress_period = pd.DataFrame({'Desk' : sp_pl_sum_min.index, 'start_date' : sp_start_date, 'end_date' : sp_end_date})

In [273]:
stress_period

,Desk,start_date,end_date
0,DESK3_A_TOTAL,2008-04-25,2009-04-14
1,DESK3_B_TOTAL,2008-04-24,2009-04-13
2,DESK3_C_TOTAL,2008-04-25,2009-04-14
3,DESK3_D_TOTAL,2008-02-13,2009-01-30
4,DESK3_E_TOTAL,2008-02-13,2009-01-30
5,DESK3_F_TOTAL,2009-01-19,2010-01-06
6,DESK3_G_TOTAL,2009-01-19,2010-01-06
7,DESK3_H_TOTAL,2009-09-24,2010-09-13
8,DESK3_I_TOTAL,2009-01-19,2010-01-06
9,DESK3_J_TOTAL,2008-08-04,2009-07-22
